# Crossed vs Nested Random Effects Models in Python

This notebook demonstrates the differences between crossed and nested random effects models using Python.

## Table of Contents
1. [Introduction](#introduction)
2. [Theoretical Background](#theoretical-background)
3. [Crossed Random Effects](#crossed-random-effects)
4. [Nested Random Effects](#nested-random-effects)
5. [Model Comparison](#model-comparison)
6. [Practical Guidelines](#practical-guidelines)
7. [Conclusion](#conclusion)

## Introduction

Random effects models are used when we have data with hierarchical or grouped structures. The choice between crossed and nested random effects depends on the structure of your data and research design.

### Key Differences:
- **Crossed**: Multiple grouping factors where levels can occur across different levels of other factors
- **Nested**: Hierarchical structure where one factor is completely contained within another

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import mixedlm
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

## Theoretical Background

### Random Effects Models

In mixed-effects models, we have:
- **Fixed effects**: Parameters that are constant across groups
- **Random effects**: Parameters that vary across groups

The general form is:
$$Y_{ij} = X_{ij}\beta + Z_{ij}b_i + \epsilon_{ij}$$

Where:
- $Y_{ij}$ is the response for observation $j$ in group $i$
- $X_{ij}\beta$ represents fixed effects
- $Z_{ij}b_i$ represents random effects
- $\epsilon_{ij}$ is the residual error

## Crossed Random Effects

### Example: Student Performance Across Teachers and Subjects

In a crossed design:
- Students can be taught by multiple teachers
- Teachers can teach multiple students
- The same teacher-student combinations can occur across different subjects

In [ ]:
def simulate_crossed_data(n_students=50, n_teachers=10, n_subjects=3, n_obs_per_combination=2):
    """
    Simulate data for crossed random effects design.
    Students are crossed with teachers (not nested).
    """
    # Create combinations
    students = [f'Student_{i:02d}' for i in range(1, n_students + 1)]
    teachers = [f'Teacher_{i:02d}' for i in range(1, n_teachers + 1)]
    subjects = [f'Subject_{i}' for i in range(1, n_subjects + 1)]
    
    data = []
    
    # Random effects
    student_effects = np.random.normal(0, 1.5, n_students)
    teacher_effects = np.random.normal(0, 1.0, n_teachers)
    subject_effects = np.random.normal(0, 0.8, n_subjects)
    
    # Generate data for each combination
    for i, student in enumerate(students):
        for j, teacher in enumerate(teachers):
            for k, subject in enumerate(subjects):
                for obs in range(n_obs_per_combination):
                    # Base score
                    base_score = 75
                    
                    # Add random effects
                    score = (base_score + 
                            student_effects[i] + 
                            teacher_effects[j] + 
                            subject_effects[k] +
                            np.random.normal(0, 2))  # residual error
                    
                    data.append({
                        'student_id': student,
                        'teacher_id': teacher,
                        'subject': subject,
                        'score': score,
                        'observation': obs + 1
                    })
    
    return pd.DataFrame(data)

# Generate crossed data
crossed_data = simulate_crossed_data(n_students=20, n_teachers=5, n_subjects=2, n_obs_per_combination=3)
print(f"Crossed data shape: {crossed_data.shape}")
print("\nFirst few rows:")
crossed_data.head(10)

In [ ]:
# Visualize the crossed structure
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Heatmap showing student-teacher combinations
pivot_crossed = crossed_data.groupby(['student_id', 'teacher_id'])['score'].mean().unstack()
sns.heatmap(pivot_crossed, annot=False, cmap='viridis', ax=axes[0])
axes[0].set_title('Crossed Design: Students × Teachers\n(Each cell represents mean score)')
axes[0].set_xlabel('Teacher')
axes[0].set_ylabel('Student')

# Box plot by teacher and subject
sns.boxplot(data=crossed_data, x='teacher_id', y='score', hue='subject', ax=axes[1])
axes[1].set_title('Score Distribution by Teacher and Subject')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Fit crossed random effects model
print("Fitting Crossed Random Effects Model...")
print("Model formula: score ~ 1 + (1|student_id) + (1|teacher_id) + (1|subject)")

# Note: In statsmodels, we fit separate random effects for each grouping factor
# For a true crossed model, we'd need to use specialized libraries like pymer4
# Here we approximate with separate models for demonstration

crossed_model_student = mixedlm("score ~ 1", crossed_data, groups=crossed_data["student_id"])
crossed_result_student = crossed_model_student.fit()

print("\nModel with Student Random Effects:")
print(crossed_result_student.summary())

## Nested Random Effects

### Example: Students Nested in Classrooms, Classrooms Nested in Schools

In a nested design:
- Students belong to specific classrooms
- Classrooms belong to specific schools
- Students never switch between classrooms
- There's a clear hierarchical structure

In [ ]:
def simulate_nested_data(n_schools=5, n_classrooms_per_school=4, n_students_per_classroom=8, n_obs_per_student=3):
    """
    Simulate data for nested random effects design.
    Students nested in classrooms, classrooms nested in schools.
    """
    data = []
    
    # Random effects at each level
    school_effects = np.random.normal(0, 2.0, n_schools)
    
    student_counter = 1
    classroom_counter = 1
    
    for school_idx in range(n_schools):
        school_id = f'School_{school_idx + 1:02d}'
        
        # Classroom effects within this school
        classroom_effects = np.random.normal(0, 1.5, n_classrooms_per_school)
        
        for classroom_idx in range(n_classrooms_per_school):
            classroom_id = f'Classroom_{classroom_counter:02d}'
            classroom_counter += 1
            
            # Student effects within this classroom
            student_effects = np.random.normal(0, 1.0, n_students_per_classroom)
            
            for student_idx in range(n_students_per_classroom):
                student_id = f'Student_{student_counter:03d}'
                student_counter += 1
                
                for obs in range(n_obs_per_student):
                    # Base score
                    base_score = 70
                    
                    # Add nested random effects
                    score = (base_score + 
                            school_effects[school_idx] + 
                            classroom_effects[classroom_idx] + 
                            student_effects[student_idx] +
                            np.random.normal(0, 1.5))  # residual error
                    
                    data.append({
                        'school_id': school_id,
                        'classroom_id': classroom_id,
                        'student_id': student_id,
                        'score': score,
                        'observation': obs + 1
                    })
    
    return pd.DataFrame(data)

# Generate nested data
nested_data = simulate_nested_data(n_schools=3, n_classrooms_per_school=3, n_students_per_classroom=6, n_obs_per_student=2)
print(f"Nested data shape: {nested_data.shape}")
print("\nFirst few rows:")
nested_data.head(10)

In [ ]:
# Visualize the nested structure
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Hierarchical structure visualization
school_means = nested_data.groupby('school_id')['score'].mean()
classroom_means = nested_data.groupby(['school_id', 'classroom_id'])['score'].mean().reset_index()

axes[0, 0].bar(school_means.index, school_means.values)
axes[0, 0].set_title('Mean Scores by School')
axes[0, 0].set_ylabel('Mean Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Classroom means within schools
for school in nested_data['school_id'].unique():
    school_data = classroom_means[classroom_means['school_id'] == school]
    axes[0, 1].scatter([school] * len(school_data), school_data['score'], s=100, alpha=0.7, label=school)

axes[0, 1].set_title('Classroom Means within Schools')
axes[0, 1].set_ylabel('Mean Score')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].legend()

# 3. Box plot showing nested structure
sns.boxplot(data=nested_data, x='school_id', y='score', hue='classroom_id', ax=axes[1, 0])
axes[1, 0].set_title('Score Distribution: Schools and Classrooms')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 4. Individual student scores
student_sample = nested_data[nested_data['school_id'].isin(['School_01', 'School_02'])]
sns.stripplot(data=student_sample, x='classroom_id', y='score', hue='school_id', ax=axes[1, 1], size=4)
axes[1, 1].set_title('Individual Student Scores by Classroom')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Fit nested random effects model
print("Fitting Nested Random Effects Model...")
print("Model formula: score ~ 1 + (1|school_id/classroom_id)")

# Create a nested grouping variable
nested_data['school_classroom'] = nested_data['school_id'] + '_' + nested_data['classroom_id']

# Fit model with classroom nested in school
nested_model = mixedlm("score ~ 1", nested_data, groups=nested_data["school_id"], 
                      re_formula="1", vc_formula={"classroom_id": "0 + C(classroom_id)"})
nested_result = nested_model.fit()

print("\nNested Random Effects Model Results:")
print(nested_result.summary())

## Model Comparison

Let's compare the key characteristics and when to use each model type.

In [ ]:
# Create a comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Variance components comparison
variance_data = {
    'Model Type': ['Crossed', 'Nested'],
    'Between-Group Variance': [1.5, 2.0],  # Example values
    'Within-Group Variance': [2.0, 1.5],
    'Residual Variance': [2.0, 1.5]
}

variance_df = pd.DataFrame(variance_data)
variance_df.set_index('Model Type')[['Between-Group Variance', 'Within-Group Variance', 'Residual Variance']].plot(
    kind='bar', ax=axes[0, 0], width=0.8)
axes[0, 0].set_title('Variance Components Comparison')
axes[0, 0].set_ylabel('Variance')
axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0, 0].tick_params(axis='x', rotation=0)

# 2. Model complexity
complexity_data = {
    'Aspect': ['Number of\nRandom Effects', 'Model\nComplexity', 'Sample Size\nNeeded'],
    'Crossed': [3, 4, 3],
    'Nested': [2, 3, 2]
}

x = np.arange(len(complexity_data['Aspect']))
width = 0.35

axes[0, 1].bar(x - width/2, complexity_data['Crossed'], width, label='Crossed', alpha=0.8)
axes[0, 1].bar(x + width/2, complexity_data['Nested'], width, label='Nested', alpha=0.8)
axes[0, 1].set_title('Model Characteristics (Relative Scale)')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(complexity_data['Aspect'])
axes[0, 1].legend()

# 3. Design structure visualization
# Crossed design representation
crossed_matrix = np.random.rand(5, 5)
im1 = axes[1, 0].imshow(crossed_matrix, cmap='Blues', aspect='auto')
axes[1, 0].set_title('Crossed Design Structure\n(Students × Teachers)')
axes[1, 0].set_xlabel('Teachers')
axes[1, 0].set_ylabel('Students')

# Nested design representation
nested_matrix = np.zeros((15, 15))
for i in range(3):
    for j in range(5):
        nested_matrix[i*5:(i+1)*5, i*5:(i+1)*5] = np.random.rand(5, 5)

im2 = axes[1, 1].imshow(nested_matrix, cmap='Greens', aspect='auto')
axes[1, 1].set_title('Nested Design Structure\n(Students within Classrooms within Schools)')
axes[1, 1].set_xlabel('Hierarchical Units')
axes[1, 1].set_ylabel('Hierarchical Units')

plt.tight_layout()
plt.show()

## Practical Guidelines

### When to Use Crossed Random Effects:
1. **Multiple grouping factors** that are not hierarchically related
2. **Factorial designs** where factors can occur in all combinations
3. **Repeated measures** across different conditions or stimuli
4. **Examples**: 
   - Students × Teachers × Subjects
   - Participants × Items in psycholinguistic studies
   - Patients × Doctors × Treatments

### When to Use Nested Random Effects:
1. **Clear hierarchical structure** in your data
2. **One grouping factor contained within another**
3. **Natural clustering** that follows organizational structure
4. **Examples**:
   - Students within Classes within Schools
   - Employees within Departments within Companies
   - Measurements within Patients within Hospitals

In [ ]:
# Decision flowchart as code
def model_selection_guide(grouping_factors):
    """
    Simple decision guide for choosing between crossed and nested models.
    """
    print("Model Selection Guide")
    print("=" * 50)
    
    questions = [
        "Can levels of Factor A occur with multiple levels of Factor B?",
        "Can levels of Factor B occur with multiple levels of Factor A?",
        "Is there a clear hierarchical relationship between factors?",
        "Are some groups completely contained within others?"
    ]
    
    print("Answer these questions about your data:")
    for i, question in enumerate(questions, 1):
        print(f"{i}. {question}")
    
    print("\nDecision Rules:")
    print("- If you answered YES to questions 1 AND 2: Use CROSSED random effects")
    print("- If you answered YES to questions 3 AND 4: Use NESTED random effects")
    print("- If you answered YES to 1&2 AND 3&4: Consider a more complex model")
    
model_selection_guide(['Factor A', 'Factor B'])

## Summary Statistics and Model Diagnostics

In [ ]:
# Summary statistics for both datasets
print("CROSSED DESIGN SUMMARY")
print("=" * 50)
print(f"Total observations: {len(crossed_data)}")
print(f"Number of students: {crossed_data['student_id'].nunique()}")
print(f"Number of teachers: {crossed_data['teacher_id'].nunique()}")
print(f"Number of subjects: {crossed_data['subject'].nunique()}")
print(f"Mean score: {crossed_data['score'].mean():.2f}")
print(f"Standard deviation: {crossed_data['score'].std():.2f}")

print("\nNESTED DESIGN SUMMARY")
print("=" * 50)
print(f"Total observations: {len(nested_data)}")
print(f"Number of schools: {nested_data['school_id'].nunique()}")
print(f"Number of classrooms: {nested_data['classroom_id'].nunique()}")
print(f"Number of students: {nested_data['student_id'].nunique()}")
print(f"Mean score: {nested_data['score'].mean():.2f}")
print(f"Standard deviation: {nested_data['score'].std():.2f}")

# ICC calculations
print("\nINTRACLASS CORRELATION COEFFICIENTS (ICC)")
print("=" * 50)

# For nested data - School level ICC
school_var = nested_data.groupby('school_id')['score'].var().mean()
total_var = nested_data['score'].var()
school_icc = school_var / total_var
print(f"School-level ICC (nested): {school_icc:.3f}")

# For crossed data - Student level ICC
student_var = crossed_data.groupby('student_id')['score'].var().mean()
total_var_crossed = crossed_data['score'].var()
student_icc = student_var / total_var_crossed
print(f"Student-level ICC (crossed): {student_icc:.3f}")

## Conclusion

### Key Takeaways:

1. **Model Selection**: The choice between crossed and nested random effects depends on your data structure and research design, not statistical preferences.

2. **Crossed Models** are appropriate when:
   - Multiple grouping factors are not hierarchically related
   - Factors can appear in all possible combinations
   - You want to account for multiple sources of random variation simultaneously

3. **Nested Models** are appropriate when:
   - There's a clear hierarchical structure in your data
   - Lower-level units are completely contained within higher-level units
   - You want to partition variance across different levels of hierarchy

4. **Practical Considerations**:
   - Nested models are often simpler to interpret
   - Crossed models can handle more complex designs but require larger sample sizes
   - Both approaches help account for non-independence in grouped data

5. **Model Validation**:
   - Always check model assumptions
   - Examine residual plots
   - Consider model fit statistics (AIC, BIC)
   - Validate with domain expertise

### Next Steps:
- Explore the R notebook for additional examples and different modeling approaches
- Try applying these concepts to your own data
- Consider more complex models with both crossed and nested effects when appropriate